# Module 3 - GraphRAG and Agents

This module has the following objectives:
- Experiment with queries for an Agent
- Define Tooling
- Create an agents with the available tools
- Chatbot for an Agent
- Text2Cypher (if we got time)

In [1]:
#!pip install graphdatascience neo4j dotenv openai langchain, langgraph, pydantic, gradio

Import our usual suspects (and some more...)

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result
from langchain.schema import HumanMessage
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.prebuilt import create_react_agent
from openai import OpenAI
from typing import List, Optional
from pydantic import BaseModel, Field, validator
import functools
from langchain_core.tools import tool
import gradio as gr
import time
from json import loads, dumps
from openai import OpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, Graph, START, END
from IPython.display import Image, display
from typing import Literal
from langgraph.checkpoint.memory import MemorySaver

## Setup

Load env variables

In [3]:
env_file = 'credentials.env'

In [4]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

Connect to neo4j db

In [5]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the connection

In [6]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,1494


Test whether we got our constraints

In [7]:
schema_result_df  = driver.execute_query(
    'show indexes',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [8]:
schema_result_df.head(100)

,id,name,state,populationPercent,type,entityType,labelsOrTypes,properties,indexProvider,owningConstraint,lastRead,readCount
0,6,chunk-embeddings,ONLINE,100.0,VECTOR,NODE,[Chunk],[embedding],vector-2.0,None,2025-05-14T14:38:01.962000000+00:00,113
1,0,index_343aff4e,ONLINE,100.0,LOOKUP,NODE,None,None,token-lookup-1.0,None,2025-05-14T14:56:08.870000000+00:00,3813
2,1,index_f7700477,ONLINE,100.0,LOOKUP,RELATIONSHIP,None,None,token-lookup-1.0,None,2025-05-14T14:56:09.496000000+00:00,104
3,4,unique_chunk,ONLINE,100.0,RANGE,NODE,[Chunk],[id],range-1.0,unique_chunk,2025-05-14T14:38:01.989000000+00:00,11035
4,2,unique_document,ONLINE,100.0,RANGE,NODE,[Document],[id],range-1.0,unique_document,2025-05-13T12:31:28.842000000+00:00,2742


In [9]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDINGS_MODEL,
    openai_api_key=OPENAI_API_KEY
)

In [10]:
embedding_model.model

'text-embedding-ada-002'

## Agents with GraphRAG

### Lets create a Retrieval agent

In [11]:
# class Skill(BaseModel):
#     """
#     Represents a professional skill or knowledge of a person.
#     """
#     name: str = Field(..., description="Sortened name of the skill")

### Retrieve Product or Customer

In [12]:
client = OpenAI()

In [13]:
retrieve_product_customer_prompt = """
As an intelligent assistant, your primary objective is to find either a customer name or product from the text submitted. 
The goal is to retrieve either the product on which someone is asking a question. Or the customer name such that we can find the products from their. 
Please only return the product name that is extracted. 

Examples:
#####
User: I got a question about my savings account. 
Assistant: Product: savings account
#####
#####
User: My name is Jan Blok and I got a question. 
Assistant: Customer: Jan Blok
#####
#####
User: What is the policy on account? 
Assistant: Need more information
#####
"""

def retrieve_product_or_customer_from_text(question):
    """Retrieve either products or customers from the text given by the user"""

    response = client.beta.chat.completions.parse(
        model=LLM,
        temperature=0,
        messages=[
            {"role": "system", "content": retrieve_product_customer_prompt},
            {"role": "user", "content": question},
        ],
#        response_format=DefinitionList,
    )
    return response.choices[0].message.content 

In [14]:
retrieve_product_or_customer_from_text("I got a question about my savings account")

'Product: savings account'

In [15]:
retrieve_product_or_customer_from_text("Can I have overdraft on my account?")

'Need more information'

In [16]:
retrieve_product_or_customer_from_text("I'm Erik Bijl, can I have overdraft on my account?")

'Customer: Erik Bijl'

### Map Products

In [17]:
def retrieve_products() -> pd.DataFrame:
    """Retrieve the products in the database. Products are specified with name. """
    return driver.execute_query(
        """
        MATCH (p:ProductType)
        RETURN p.name as name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df(),
    )['name'].tolist()

In [18]:
retrieve_products()

['SpaarRekening',
 'DirectRekening',
 'Kortlopende Reis',
 'BeleggersRekening',
 'RaboBusiness Banking']

In [19]:
map_products_prompt = """
As an intelligent assistant, your primary objective is to map a product name to product names in the database.

Examples:
#####
Product: savings account. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']
Assistant: Product: SpaarRekening
#####
#####
Product: Direct Rekening. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']Assistant: Customer: Jan Blok
Assistant: Product: DirectRekening

#####
#####
Product: Reis verzekering. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']Assistant: Customer: Jan Blok
Assistant: Product: Kortlopende Reis
#####
"""

def map_product_to_database_products(product) -> str:
    """Map products from the user question to the actual products in the database."""

    response = client.beta.chat.completions.parse(
        model=LLM,
        temperature=0,
        messages=[
            {"role": "system", "content": map_products_prompt},
            {"role": "user", "content": "Product: " + product},
            {"role": "user", "content": "Database Products: " + str(retrieve_products())},
            
        ],
#        response_format=DefinitionList,
    )
    return response.choices[0].message.content 

In [20]:
map_product_to_database_products('savings account')

'Product: SpaarRekening'

In [21]:
def retrieve_document_from_product(product_name) -> pd.DataFrame:
    """Retrieve the documents of products in the database. Products are specified with their name. """
    return driver.execute_query(
        """
        MATCH (p:ProductType)<-[:RELATED_TO]-(d:Document)
        WHERE LOWER(p.name) = LOWER($product_name)
        RETURN d.file_name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        product_name = product_name,
        result_transformer_= lambda r: r.to_df(),
    ).iloc[0]['d.file_name']

In [22]:
retrieve_document_from_product('SpaarRekening')

'Rabo SpaarRekening 2020.pdf'

### GraphRAG

In [23]:
def get_context_graphrag(document, search_prompt):

    query_vector = embedding_model.embed_query(search_prompt)
    
    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 30, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        MATCH (d:Document {file_name: $document})<-[:PART_OF]-(chunk)
        WITH score, d, chunk LIMIT 5
        RETURN score, d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
       """
    results = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        document = document,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )

    chunk_ids = list(set(results['chunk_id'].to_list()))

    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=4)

    definition_query = """    
        MATCH (c:Chunk)-[:MENTIONS]->(d:Definition)
        WHERE c.id in $chunk_ids
        RETURN DISTINCT d.term as term, d.description as description
    """
    results = driver.execute_query(
        definition_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        chunk_ids=chunk_ids,
        result_transformer_= lambda r: r.to_df()
    )
    results = results.to_json(orient="records")
    parsed = loads(results)
    definitions = dumps(parsed, indent=4)
    return context, definitions

In [24]:
def generate_prompt_graphrag(search_prompt, context, definitions):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    Respond in English. 

    The question is the following: 
    {search_prompt}
    
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}

    The definitions are the following: 
    {definitions}
    
    Please end your message with listing your sources with file name and page number. 
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context, definitions=definitions)
    return theprompt

In [25]:
def perform_search_in_document(document, search_prompt) -> pd.DataFrame:
    """Peform a search in the document to search relevant text and definitions to answer a user question. The document first needs to be determined before a search should be performed."""

    context, definitions = get_context_graphrag(document, search_prompt)
    
    return context, definitions

In [26]:
def answer_question(state):
    context, definitions = perform_search_in_document(state['document'], state['question'])
    theprompt = generate_prompt_graphrag(state['question'], context, definitions)
    llm(theprompt.to_messages()).pretty_print()

In [27]:
llm = ChatOpenAI(model_name=LLM, temperature=0)

In [28]:
answer_question({'document': "Rabo SpaarRekening 2020.pdf", 'question': "What are the rules for shared savings account?"})

/var/folders/l0/vn55w75s41703t1w811dtf080000gp/T/ipykernel_54575/2830703966.py:4: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(theprompt.to_messages()).pretty_print()


================================== Ai Message ==================================

The rules for a shared savings account, also known as a joint account, with Rabobank are as follows:

1. **Joint Account Definition**: If the savings account has multiple account holders, it is considered a joint account. This is only different if it has been agreed with the bank that it is a joint-and account.

2. **Communication**: Rabobank only needs to inform one account holder, either in writing or electronically. It is the responsibility of the informed account holder to immediately share any information with the other account holders. All account holders are bound by the information provided to one of them, even if they do not live at the same address.

3. **Notifications**: If one account holder informs the bank or makes a notification, it is assumed that this is done on behalf of all account holders.

4. **Usage Restrictions**: In the event of bankruptcy, legal debt restructuring, or seizure, non

In [29]:
class State(TypedDict):
    input: str
    question: str
    product: str
    customer: str
    document: str

from langgraph.types import interrupt, Command

# no-op node that should be interrupted on
def human_feedback(state: State):
    pass


def retrieve_product_customer(state):
    result = retrieve_product_or_customer_from_text(state['input'])
    if 'Customer: ' in result:
        customer = result.split('Customer: ')[1]
        print('It looks like you are the following customer: ' + customer)
        state['customer'] = customer
    elif 'Product: ' in result:
        product = result.split('Product: ')[1]
        print('It sounds like you got a question on the following product: ' + product)
        state['product'] = product
    return state

def user_feedback(state):
    print("I couldn't find information to start the flow. Could you specify a product on which you have questions or your customer name?")
    return state 
    
def map_product_to_db(state):
    result = map_product_to_database_products(state['product'])
    product = result.split('Product: ')[1]
    state['product'] = product
    print("I have found the following product in the database: " + product)
    return state

def retrieve_document(state):
    result = retrieve_document_from_product(state['product'])
    state['document'] = result
    print("I found the following document on the product: " + result)
    return state

def tools_condition(state) -> Literal["user_feedback", "map_product"]: 
    if (state.get('product') is None):
        return "user_feedback"
    else: 
        return "map_product"

# Build graph
builder = StateGraph(State)
builder.add_node("retrieve_product_customer", retrieve_product_customer)
builder.add_node("user_feedback", user_feedback)
builder.add_node("map_product", map_product_to_db)
builder.add_node("retrieve_document", retrieve_document)
builder.add_node("human_feedback", human_feedback)
builder.add_node("answer_question", answer_question)


# Logic
builder.add_edge(START, "retrieve_product_customer")
builder.add_conditional_edges("retrieve_product_customer", tools_condition)
builder.add_edge("user_feedback", END)
builder.add_edge("map_product", "retrieve_document")
builder.add_edge("retrieve_document", "human_feedback")
builder.add_edge("human_feedback", "answer_question")
builder.add_edge("answer_question", END)

memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory, interrupt_before=["human_feedback"])

In [30]:
# graph.get_graph().draw_mermaid_png()

In [31]:
graph.get_graph().print_ascii()

                   +-----------+                   
                   | __start__ |                   
                   +-----------+                   
                          *                        
                          *                        
                          *                        
           +---------------------------+           
           | retrieve_product_customer |           
           +---------------------------+           
                 ...            ...                
               ..                  ...             
             ..                       ..           
   +-------------+                      ..         
   | map_product |                       .         
   +-------------+                       .         
          *                              .         
          *                              .         
          *                              .         
+-------------------+                    .         
| retrieve_d

In [32]:
thread_no = 0

In [59]:
initial_input = {"input" : "I got a question on my savings account, what are the rules for a shared account?"}

thread  = {"configurable": {"thread_id": "25"}}
for event in graph.stream(initial_input, thread, stream_mode="values"):
    print(event)

print(graph.get_state(thread))

user_input = input(f"Please raise your question on {graph.get_state(thread).values['product']}: ")

graph.update_state(thread, {"question": user_input}, as_node="human_feedback")

graph.get_state(thread).next

for event in graph.stream(None, thread, stream_mode="values"):
    print(event)

{'input': 'I got a question on my savings account, what are the rules for a shared account?'}
It sounds like you got a question on the following product: savings account
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'savings account'}
I have found the following product in the database: SpaarRekening
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening'}
I found the following document on the product: Rabo SpaarRekening 2020.pdf
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf'}
StateSnapshot(values={'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf'}, next=('human_feedback',), config={'configurable': {'thread_id': '25', 'checkpoint_ns': '', 

TypeError: 'dict' object is not callable

In [35]:
initial_input = {"input" : "My name is Eva Meijer, I got a question what my interest rate is on my savings account. "}

thread  = {"configurable": {"thread_id": "2"}}
for event in graph.stream(initial_input, thread, stream_mode="values"):
    print(event)

{'input': 'My name is Eva Meijer, I got a question what my interest rate is on my savings account. '}
It looks like you are the following customer: Eva Meijer
{'input': 'My name is Eva Meijer, I got a question what my interest rate is on my savings account. ', 'customer': 'Eva Meijer'}
I couldn't find information to start the flow. Could you specify a product on which you have questions or your customer name?
{'input': 'My name is Eva Meijer, I got a question what my interest rate is on my savings account. ', 'customer': 'Eva Meijer'}


In [55]:
import gradio as gr
import io
import contextlib

threads = {}

def chat_interface(user_input, chat_history=[], thread_id="default"):
    # Reuse or initialize thread
    if thread_id not in threads:
        threads[thread_id] = {
            "configurable": {"thread_id": thread_id},
            "step": "initial"
        }

    thread = threads[thread_id]

    # Capture all print output from graph steps
    f = io.StringIO()
    with contextlib.redirect_stdout(f):
        if thread["step"] == "initial":

            initial_input = {"input": user_input}
            for _ in graph.stream(initial_input, thread, stream_mode="values"):
                pass

            thread["step"] = "waiting_for_question"
            output_text = f.getvalue().strip()

            # Determine if human input is needed
            next_node = graph.get_state(thread).next
            if next_node == "human_feedback":
                print("HELLOOOO")
                product = graph.get_state(thread).values.get("product", "the product")
                assistant_message = f"{output_text}\nPlease clarify your question on **{product}**."
                print(f"Please clarify your question on {product}")
                return "", chat_history + [(user_input, assistant_message)]

            return "", chat_history + [(user_input, output_text)]

        elif thread["step"] == "waiting_for_question":
            graph.update_state(thread, {"question": user_input}, as_node="human_feedback")
            thread["step"] = "done"

            for _ in graph.stream(None, thread, stream_mode="values"):
                pass

            output_text = f.getvalue().strip()
            final_state = graph.get_state(thread)
            final_answer = final_state.values.get("answer", "[No answer generated]")

            assistant_message = f"{output_text}\n\n🤖 **Answer:** {final_answer}"
            return "", chat_history + [(user_input, assistant_message)]

        else:
            return "", chat_history + [(user_input, "⚠️ Conversation is complete. Please reset to start again.")]

    return "", chat_history + [(user_input, "[Unexpected error occurred]")]


In [57]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message")
    send_btn = gr.Button("Send")
    reset_btn = gr.Button("Reset")

    def respond(message, chat_history):
        return chat_interface(message, chat_history)

    def reset():
        threads.clear()
        return []

    send_btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    reset_btn.click(reset, outputs=[chatbot])

demo.launch()

/Users/erikbijl/anaconda3/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


### Retrieve Products in the System

In [41]:
def retrieve_products() -> pd.DataFrame:
    """Retrieve the products in the database. Products are specified with name. """
    return driver.execute_query(
        """
        MATCH (p:ProductType)
        RETURN p.name as name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df(),
    )

In [29]:
retrieve_products() 

,name
0,SpaarRekening
1,DirectRekening
2,Kortlopende Reis
3,BeleggersRekening
4,RaboBusiness Banking


In [167]:
def retrieve_document_from_product(product_name) -> pd.DataFrame:
    """Retrieve the documents of products in the database. Products are specified with their name. """
    return driver.execute_query(
        """
        MATCH (p:Product)-[:OF_TYPE]->(:ProductType)<-[:RELATED_TO]-(d:Document)
        WHERE LOWER(p.name) = LOWER($product_name)
        RETURN d.file_name LIMIT 1
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        product_name = product_name,
        result_transformer_= lambda r: r.to_df(),
    )

In [168]:
retrieve_document_from_product('Kortlopende Reis Product')

,d.file_name
0,Interpolis Short-Term Travel Insurance.pdf


### GraphRAG

## Setting up the Agent

In [28]:
llm = ChatOpenAI(model_name=LLM, temperature=0)

In [29]:
tools = [
    retrieve_products, 
    perform_search_in_document,
    retrieve_document_from_product,
    retrieve_products_of_customer,
    map_product_to_database_products,
    retrieve_product_or_customer_from_text
]

llm_with_tools = llm.bind_tools(tools)

NameError: name 'retrieve_products_of_customer' is not defined

In [176]:
from langchain_core.prompts import PromptTemplate

template = '''
    As an intelligent assistant, your primary objective is to find answeres to questions on products. 
    The questions can be answered by looking in the documentation of these products (context_search).
    However, before finding the right documentation we need to find the products on which the questions are asked. Find this out first.
    If this is not clear kindly ask the customer that you are there to help and you need either a customer name or product.
    You can find the related documents from the user question directly or you can find it from a customer name. 
    In case you have found the customer name, just query the database with the client name to find the products the client is having questions about. 
    You can answer only on the products that are actually in the database. So, map the products to the ones in the database.
'''

prompt = PromptTemplate.from_template(template)

In [183]:
agent_executor = create_react_agent(model=llm, tools=tools, prompt=prompt)

In [184]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

In [185]:
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='3bf26016-ecd4-4974-8716-8388d7054a87'),
 AIMessage(content='Please provide either the name of the product or the customer name so I can assist you with your question.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 397, 'total_tokens': 420, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'finish_reason': 'stop', 'logprobs': None}, id='run-12bf0b2a-50b7-4fa3-b989-e902b409577d-0', usage_metadata={'input_tokens': 397, 'output_tokens': 23, 'total_tokens': 420, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

#### Run some examples! 

In [196]:
def ask_to_agent(question):
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content=question)]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()

In [197]:
question = "Can I have overdraft on my account?"

In [198]:
ask_to_agent(question)

================================ Human Message =================================

Can I have overdraft on my account?
================================== Ai Message ==================================

Please provide me with either the name of the product or the customer name so I can assist you better.


## Chatbot

Now create a chatbot with the agent providing the responses

In [199]:
def user(user_message, history):
    if history is None:
        history = []
    history.append({"role": "user", "content": user_message})
    return "", history

def get_answer(history):
    steps = []
    full_prompt = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])
    
    for step in agent_executor.stream(
            {"messages": [HumanMessage(content=full_prompt)]},
            stream_mode="values",
    ):
        step["messages"][-1].pretty_print()
        steps.append(step["messages"][-1].content)
    
    return steps[-1]

def bot(history):
    bot_message = get_answer(history)
    history.append({"role": "assistant", "content": ""})

    for character in bot_message:
        history[-1]["content"] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot on a Graph",
        avatar_images=[
            "https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg",
            "https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"
        ],
        type="messages", 
    )
    msg = gr.Textbox(label="Message")
    clear = gr.Button("Clear")

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot], chatbot
    )

    clear.click(lambda: [], None, chatbot, queue=False)

demo.queue()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://124eec5ce9b2cdf8b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


If you want to have the light-mode for the chatbot paste the following after the URL: /?__theme=light